## Data Wrangling and Analysis of Open Street Map Data of Columbus Ohio

The purpose of this project was to analyze a portion of the user submitted map system, Open Street Map (https://www.openstreetmap.org/) for a given area using SQLlite. Since Open Street Map is user submitted, some data cleaning was needed before converting the OSM file to csv's and entering them into a database. The area chosen for this project was Columbus, OH. and the area data can be downloaded at https://mapzen.com/data/metro-extracts/ and entering Columbus, OH. as the extract. You can see the size of the Columbus OSM file below along with the CSV files used for tables extrated from it in the output of the following cell: 

In [1]:
import xml.etree.ElementTree as ET                     # initalizing all the libraries that were used
import re, os, csv, codecs, cerberus, schema, sqlite3  # for this project
OSM_FILE = "Columbus_OH_Map.osm"# downloaded from Map Zen, unmodified map of Columbus, OH search. 
db = sqlite3.connect("Columbus_OSM") #name of local database
c = db.cursor()
def get_row_count(table): #helper function that is used to return the number of entries in a given sql table
    return c.execute("SELECT count(*) from %s" % table).fetchone()[0] # tuple output                              
print """size of Columbus_OSM_file: {} bytes
size of nodes csv: {} bytes                   with                              {} entries (lines)
size of nodes_tags csv: {} bytes               with                              {} entries (lines)
size of ways csv: {} bytes                     with                              {} entries (lines)
size of ways_tags csv: {} bytes               with                              {} entries (lines)
size of ways_nodes csv: {} bytes              with                              {} entries (lines)""".format(os.path.getsize(OSM_FILE), 
os.path.getsize('nodes.csv'),get_row_count('nodes'), os.path.getsize('nodes_tags.csv'), get_row_count('nodes_tags'), os.path.getsize('ways.csv'), get_row_count('ways'), os.path.getsize('ways_tags.csv'),
get_row_count('ways_tags'), os.path.getsize('ways_nodes.csv'), get_row_count('ways_nodes'))
db.close()

size of Columbus_OSM_file: 150833754 bytes
size of nodes csv: 57188586 bytes                   with                              672091 entries (lines)
size of nodes_tags csv: 1370144 bytes               with                              37032 entries (lines)
size of ways csv: 4727668 bytes                     with                              78162 entries (lines)
size of ways_tags csv: 13480752 bytes               with                              373655 entries (lines)
size of ways_nodes csv: 19591389 bytes              with                              799209 entries (lines)


As you can see, the size of the (uncompressed) Columbus OSM file is fairly large, about 150 MB and the nodes and ways roots contain about 2 million entries. The number of lines was obtained by looking at the number of rows in the final sql table, the schema for those tables can be found here https://gist.github.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f. As noted, since this large file is all user generated, there is bound to be some messy data and in my inital screening I found a few problem areas: phone numbers, street names and restaurant cuisine designations.

### Phone Numbers

Open Street map allows user to enter a tag for a phone number associated with a root node, the problem is that there is no standardization of the format. Some users submit numbers in formats like 1 + (xxx) xxx - xxxx or (xxx)xxxxxxx, etc. I was interested in knowing the breakdown of area codes in Columbus so this had to be clean this up. I used the following function when converting the OSM file to CSVs to do so. 

In [2]:
def new_phone_num(number):
    """take a phone number in any format and return it as (xxx) 123-4567 or if there's an ext (xxx) 123-4567 x 1234"""
    number.split() # splitting because itll be easier to deal as a case by case basis
    new = ""
    for value in number:
        try: 
            int(value)
            new += str(value)
        except:
            pass
    if new[0] == "1" and new[:2] != '18': # getting rid of 1 digit but keeping for 1800/1888 numbers
        new = new[1:]
    if len(new) == 10: # for numbers without an extension
        new = '({}) {}-{}'.format(new[0:3], new[3:6], new[6:])
    elif new[:2] == "18": # for numbers starting with 1800
        new = '({}) {}-{}'.format(new[0:4], new[4:7], new[7:])
    else:
        new = '({}) {}-{} x {}'.format(new[0:3], new[3:6], new[6:10], new[10:]) #numbers with extensions
        if len(new[14:]) >= 10: #in case there is two numbers listed
            new = '{} / {}'.format(new[:14], new_phone_num(new[16:]))
    return new

Now that the phone numbers are cleaned up, I can use sql lite to analyze the breakdown

In [3]:
db = sqlite3.connect("Columbus_OSM") #name of local database
c = db.cursor()
print c.execute("""SELECT SUBSTR(value, 0, 5) as area_code, count(SUBSTR(value, 0, 5)) as count
                from nodes_tags where key = "phone" group by area_code order by count desc;""").fetchall()
db.close()

[(u'(614', 189), (u'(740', 2), (u'(164', 1), (u'(800', 1), (u'(866', 1), (u'614-', 1)]


As the above list shows, the majority of buisiness listed have a 614 area code which is Columbus' primary one. Based on personal knowledge though, I know that there should be far more 800/866 area codes for local buisnesses but since this is user generated, they are probably not logging those companies. The 164 area code is strange though, that may be from a typo, to confirm this, I can look at the entry associated with this number:

In [4]:
db = sqlite3.connect("Columbus_OSM") #name of local database
c = db.cursor()
id = c.execute("""SELECT id from nodes_tags where SUBSTR(value, 0, 5) = '(164';""").fetchone()[0]
print c.execute('select key, value from nodes_tags where id = %s' %id).fetchall()
db.close()

[(u'name', u'Chef O Nette'), (u'phone', u'(164) 488-8444'), (u'amenity', u'restaurant'), (u'smoking', u'no'), (u'website', u'http://www.chefonette.com/'), (u'takeaway', u'yes'), (u'city', u'Upper Arlington'), (u'state', u'Ohio'), (u'street', u'Tremont Center'), (u'postcode', u'43221'), (u'opening_hours', u'7:30 am - 8 pm M-Sa'), (u'housenumber', u'2090')]


A Google search of Chef O Nette shows that the phone nubmer is actually (614) 488-8444, so my suspicion that it was a typo was correct . Ideally, this should of been caught before transfering to the database but it can easily be corrected in the manner that I used for cleaning up street types and cuisines.

## Street Types

Like phone numbers, there was no consistency with how street names were added. Sometimes the address type my be listed as Ave., Ave, avenue, etc. To make things consistent, all street names should have the type full written out, so Fake 123 St. should be listed as Fake 123 Street. This is easily accomplished by making a dictionary of all all the abbreviations in the data set and using a re expresion to pick out the street type and convert it:

In [5]:
mapping = { "St": "Street", "St.": "Street","Rd.": "Road","Ave" : "Avenue", 'Dr' : "Drive", "Dr." : "Drive", 'N' : 'North', 
          'Rd' : 'Road', 'Rd.' : 'Road', 'St' : 'Street', 'St.' : 'Street', "Blvd" : "Boulevard", 'Pkwy' : 'Parkway', 
          'S' : 'South', 'S.': 'South', 'SW':'Southwest'}
def update_street(name, mapping):
    street = street_type_re.search(name)
    last = street.group() 
    if last in mapping:
        name = re.sub(last, mapping[last], name)
    return name


Now, when converting the file to CSVs, the street names can be easily converted using the update_street function. 

## Cuisine

The same process was performed with the node tag 'cuisine' as street types. Since the output from the audit is fairly manageable, the original data audit is shown below:

In [6]:
def get_element(osm_file, tags):
    """Yield element if it is the right type of tag
    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
def audit_data(term):
    """function to audit a specific tag"""
    tag_dict = {}
    for event, elem in enumerate(get_element(OSM_FILE,('way', 'node'))):
        if elem.tag == "node":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == term:
                    if tag.attrib['v'].lower() not in tag_dict:
                        tag_dict[tag.attrib['v'].lower()] = 1
                    else:
                        tag_dict[tag.attrib['v'].lower()] += 1
    print sorted(tag_dict)                
audit_data('cuisine')

['american', 'american;mexican', 'asian', 'asian_fusion', 'bagel', 'barbecue', 'bavarian;ice_cream', 'beer;wings', 'breakfast', 'burger', 'burger,_shakes', 'burrito,_taco, salad, chips', 'cafe', 'caribbean', 'chicken', 'chicken_', 'chinese', 'chinese;vietnamese', 'coffee_shop', 'deli', 'dessert', 'diner', 'donuts', 'french', 'frozen_yogurt', 'fusion', 'fusion;coffee', 'german', 'greek', 'hawaiian', 'ice_cream', 'ice_cream,_frozen_yogurt', 'indian', 'international', 'italian', 'italian._pizza,_bbq,_regional', 'japanese', 'japanese_steakhouse', 'japanese_sushi_hibachi', 'juice', 'lebanese-american food', 'lebanese_/_mideastern', 'mediterranean', 'mexican', 'middle_eastern', 'nuts;candy', 'pasta', 'pizza', 'regional', 'sandwich', 'seafood', 'smoothies', 'spanish', 'steak', 'subs', 'sushi', 'tea', 'turkish', 'vegan', 'vietnamese', 'vietnamese,_cambodian', 'wings', 'yogurt']


Looking at the types of cuisines, it's fairly obvious that there is no real cohesion to the values. To clean this up, a helper function was definied to rename the values to some standard. It should be noted that this standard is fairly ambiguous and might not be  entirely accurate with each classification but for the purpose of this exercise it'll suffice.b

In [7]:
cuisine_map = {'bavarian;ice_cream' : 'ice_cream', 'asian_fusion': 'asian', 'sushi' : 'japanese', 'beer;wings' : 'wings',
               'burger,_shakes' : 'burger', 'lebanese-american food': 'lebanese-american', 'japanese_sushi_hibachi' : 'japanese',
              'chicken_' : 'chicken', 'ice_cream,_frozen_yogurt' : 'ice_cream', 'chinese;vietnamese': 'asian', 'japanese_steakhouse'
              : 'japanese', 'lebanese_/_mideastern': 'middle_eastern','vietnamese,_cambodian' : 'vietnamese',
              'italian._pizza,_bbq,_regional':'italian', 'frozen_yogurt' : 'yogurt', 'fusion;coffee' : 'coffee',
               'american;mexican' : 'american-mexican', 'burrito,_taco, salad, chips': 'buritto-taco', 'nuts;candy': 'nuts-candy',
              'coffee_shop' : 'coffee'}
def update_cuisine(name, mapping):
    # YOUR CODE HERE
    name = name.lower() #updating to all lowercase
    if name in mapping:
        name = mapping[name]
    return name

So by using the above function and dictionary, the values are cleaned up to the following:

In [8]:
def audit_cuisine_helper(term):
    restaurants = {}
    """function to audit a specific tag"""
    #osm_file = open(osmfile, "r")
    for event, elem in enumerate(get_element(OSM_FILE,('way', 'node'))):
        if elem.tag == "node":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == term:
                    if update_cuisine(tag.attrib['v'], cuisine_map) not in restaurants:
                        restaurants[update_cuisine(tag.attrib['v'], cuisine_map)] = 1
                    else:
                        restaurants[update_cuisine(tag.attrib['v'], cuisine_map)] += 1
    print sorted(restaurants.keys())

audit_cuisine_helper('cuisine')

['american', 'american-mexican', 'asian', 'bagel', 'barbecue', 'breakfast', 'burger', 'buritto-taco', 'cafe', 'caribbean', 'chicken', 'chinese', 'coffee', 'deli', 'dessert', 'diner', 'donuts', 'french', 'fusion', 'german', 'greek', 'hawaiian', 'ice_cream', 'indian', 'international', 'italian', 'japanese', 'juice', 'lebanese-american', 'mediterranean', 'mexican', 'middle_eastern', 'nuts-candy', 'pasta', 'pizza', 'regional', 'sandwich', 'seafood', 'smoothies', 'spanish', 'steak', 'subs', 'tea', 'turkish', 'vegan', 'vietnamese', 'wings', 'yogurt']


Now  that that's cleaned up, we can analyze the most popular cuisines of Columbus!

In [9]:
db = sqlite3.connect("Columbus_OSM") #name of local database
c = db.cursor()
print c.execute("""SELECT value, count(value) as count FROM nodes_tags WHERE key = 'cuisine'
GROUP BY value ORDER BY count DESC;""").fetchall()
db.close()

[(u'burger', 43), (u'mexican', 35), (u'pizza', 35), (u'sandwich', 29), (u'american', 22), (u'coffee_shop', 21), (u'chinese', 18), (u'italian', 17), (u'asian', 16), (u'japanese', 12), (u'ice_cream', 11), (u'chicken', 8), (u'greek', 7), (u'indian', 5), (u'regional', 4), (u'middle_eastern', 3), (u'vietnamese', 3), (u'wings', 3), (u'barbecue', 2), (u'breakfast', 2), (u'french', 2), (u'german', 2), (u'international', 2), (u'mediterranean', 2), (u'pasta', 2), (u'subs', 2), (u'yogurt', 2), (u'american;mexican', 1), (u'bagel', 1), (u'burrito,_taco, salad, chips', 1), (u'cafe', 1), (u'caribbean', 1), (u'coffee', 1), (u'deli', 1), (u'dessert', 1), (u'diner', 1), (u'donuts', 1), (u'fusion', 1), (u'hawaiian', 1), (u'juice', 1), (u'lebanese-american', 1), (u'nuts;candy', 1), (u'seafood', 1), (u'smoothies', 1), (u'spanish', 1), (u'steak', 1), (u'tea', 1), (u'turkish', 1), (u'vegan', 1)]


Looks like burgers, mexican, and pizza is pretty popular! Seeing this data then makes me wonder if restaurants are spread through out the city. One way to check would be to group the count of restaurants in each area code.

In [10]:
db = sqlite3.connect("Columbus_OSM") 
c = db.cursor()
print c.execute("""SELECT ways_tags.value as area_code, COUNT(ways_tags.value) as count FROM nodes_tags, ways_tags, ways_nodes
ON nodes_tags.id = ways_nodes.node_id AND ways_nodes.id = ways_tags.id WHERE ways_tags.key = 'postcode' and nodes_tags.value = 'restaurant'
GROUP BY area_code ORDER BY count DESC;""").fetchall()
db.close()

[(u'43026', 1), (u'43212', 1)]


According to this, it appears that a lot of data in the table is actually incomplete! We should check this however:

In [11]:
db = sqlite3.connect("Columbus_OSM") 
c = db.cursor()
print "These are the way id's:"
print c.execute("""SELECT ways_nodes.id FROM nodes_tags, ways_nodes
ON nodes_tags.id = ways_nodes.node_id WHERE nodes_tags.value = 'restaurant'
limit  20 """).fetchall()
print "84597718 information:"
print c.execute("select * From ways_tags where id = 84597718").fetchall()
print "104345466 information:"
print c.execute("select * From ways_tags where id = 104345466").fetchall()
print "167031493 information:"
print c.execute("select * From ways_tags where id = 167031493").fetchall()
db.close()

These are the way id's:
[(84597718,), (104345466,), (167031493,)]
84597718 information:
[(84597718, u'shop', u'mall', u'regular'), (84597718, u'source', u'Yahoo', u'regular'), (84597718, u'building', u'shop', u'regular')]
104345466 information:
[(104345466, u'city', u'Hilliard', u'addr'), (104345466, u'state', u'OH', u'addr'), (104345466, u'street', u'Main Street', u'addr'), (104345466, u'country', u'US', u'addr'), (104345466, u'postcode', u'43026', u'addr'), (104345466, u'interpolation', u'even', u'addr')]
167031493 information:
[(167031493, u'building', u'yes', u'regular'), (167031493, u'city', u'Columbus', u'addr'), (167031493, u'street', u'Yard Street', u'addr'), (167031493, u'postcode', u'43212', u'addr'), (167031493, u'housenumber', u'775', u'addr')]


As we first suspected, there's a lot of missing information! Only 3 restaurants have subsequent way_tags information! This throws into question a lot of our analysis using Open Street Maps as a sole source; are any trends we're seeing becuase of incomplete data? This makes me wonder how active Open Street Map is? Maybe usership has dropped off...

In [12]:
db = sqlite3.connect("Columbus_OSM") 
c = db.cursor()
print c.execute("""SELECT SUBSTR(timestamp,0,5) as date, count(SUBSTR(timestamp,0,5)) as count FROM nodes GROUP by date
ORDER BY count DESC """).fetchall()
db.close()

[(u'2016', 171247), (u'2009', 166865), (u'2010', 102548), (u'2017', 55853), (u'2015', 55127), (u'2013', 45044), (u'2012', 40188), (u'2014', 27682), (u'2011', 6815), (u'2008', 722)]


Looks like activity had a glut for a a few years but picked back up recently, so that doesn't explain much. Maybe our data problem has something to do with usership?

In [13]:
db = sqlite3.connect("Columbus_OSM") 
c = db.cursor()
print c.execute("""SELECT user, count(user) as count FROM nodes GROUP by user ORDER BY count DESC limit 20""").fetchall()
db.close()

[(u'woodpeck_fixbot', 195670), (u'Nimbalo', 108949), (u'Vid the Kid', 71019), (u'TIGERcnl', 21358), (u'Johnny Mapperseed', 20914), (u'Rub21', 16772), (u'S_H', 16243), (u'TimC', 15041), (u'ChrisSmit', 13595), (u'Dublin Ohio GIS', 12454), (u'ncmh91', 12414), (u'AndrewSP37', 11334), (u'St-Motel', 11278), (u'Conor Hayes', 10850), (u'Ruffigan', 9509), (u'Minh Nguyen', 7417), (u'morgankevinj', 6508), (u'PHerison', 4831), (u'union4', 4477), (u'AndrewSnow', 4177)]


Looks like the bulk of the data is coming from a few sources. If they're not updating properly, that could be the source of the incomplete data. That was specifically for nodes data though, what about way?:

In [16]:
db = sqlite3.connect("Columbus_OSM") 
c = db.cursor()
print c.execute("""SELECT user, count(user) as count FROM ways GROUP by user ORDER BY count DESC limit 20""").fetchall()
db.close()

[(u'bot-mode', 15963), (u'Nimbalo', 11066), (u'Vid the Kid', 8943), (u'S_H', 3102), (u'Johnny Mapperseed', 3012), (u'ChrisSmit', 2156), (u'Rub21', 2128), (u'ncmh91', 1690), (u'Conor Hayes', 1677), (u'St-Motel', 1675), (u'DaveHansenTiger', 1649), (u'AndrewSP37', 1603), (u'morgankevinj', 1306), (u'Ruffigan', 1282), (u'PHerison', 1054), (u'TimC', 1045), (u'Minh Nguyen', 978), (u'Nate_Wessel', 925), (u'NE2', 903), (u'AndrewSnow', 869)]


Not a huge difference here other than it looks like two different bots are making the majority of updates for their respective area. This could support the idea that if the bots aren't updating all the information, or making it available to the other bot, we would have a lot of missing data.

## Conclusions

Overall, it appears that open street map is a valuable resource but shouldn't be considered complete. It makes me realize how much of a huge endeavor something like Google Maps is, the resources that are required to make an accurate map on a worldwide scale. 

As for suggestions to improve the project, I would suggest coming up with an algorithm that identifies problem areas and notifies active users about them. It was called out earlier that the way root is incomplete but other areas are certainly missing information as well For example, there are only about 200 phone numbers listed in nodes. There are certainly more than 200 nonresidental phone numbers in Columbus. 

A point though is that we did not discuss in the project is maybe that this "meta" data isn't the focus of Open Street Maps, maybe they only really care about the maps sections, which is understandable. For a future project, a good idea would be to audit there map and location data against some known standard. 

## Supplementary

The following, for reference, is the code to convert the OSM_files to CSV:

In [ ]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"
SCHEMA = schema.schema
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for item in NODE_FIELDS:
            node_attribs[item] = element.attrib[item]
        for item in element.iter('tag'):
            item_elem = {}
            if PROBLEMCHARS.search(item.attrib['k']):
                return
            item_elem['id'] = node_attribs['id']
            if item.attrib['k'] == 'phone':
                item_elem['value'] = new_phone_num(item.attrib['v'])
            elif item.attrib['k'] == 'cuisine':
                item_elem['value'] = update_cuisine(item.attrib['v'], cuisine_map)
            else:
                item_elem['value'] = item.attrib['v']
            if LOWER_COLON.search(item.attrib['k']):
                type = LOWER_COLON.search(item.attrib['k']).group()
                split = type.split(':', 1)
                item_elem['key'] = split[1]
                item_elem['type'] = split[0]
            else:
                item_elem['type'] = 'regular'
                item_elem['key'] = item.attrib['k']
            if item_elem['type'] == 'street':
                item_elem['value'] = update_street(item_elem['value'], mapping)
            tags.append(item_elem)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        count = 0
        for item in WAY_FIELDS:
            way_attribs[item] = element.attrib[item]
        for item in element.iter('tag'):
            item_elem = {}
            if PROBLEMCHARS.search(item.attrib['k']):
                return
            item_elem['id'] = way_attribs['id']
            item_elem['value'] = item.attrib['v']
            if LOWER_COLON.search(item.attrib['k']):
                type = item.attrib['k']
                split = type.split(':', 1)
                item_elem['key'] = split[1]
                item_elem['type'] = split[0]
            else:
                item_elem['type'] = 'regular'
                item_elem['key'] = item.attrib['k']
            if item_elem['type'] == 'street':
                item_elem['value'] = update_street(item_elem['value'], mapping)
            tags.append(item_elem)
        for item in element.iter('nd'):
            way_elem = {}
            way_elem['id'] = way_attribs['id']
            way_elem['node_id'] = item.attrib['ref']
            way_elem['position'] = count
            count +=1
            way_nodes.append(way_elem)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_FILE, validate=True)
